In [3]:
%run GtfProcessor.ipynb
import pandas as pd 
import numpy as np
import sys
import itertools
from copy import deepcopy
import copy
import os

In [4]:
### delete all the LNCRNA genes that have overlapping exon on the same strand with protein coding gene. 
#The file is strand_LnRNA_ExonOverlap_ProteinCodingExons
def deleteLncRNAGenesOverlappingSameExonsSameStrand(senseExonOverlapBedFile,sameStrandDeletedGenesGTFFile,cleanedSameStrandExonsGTFFile):
    senseExonOverlapBed = pd.read_csv(senseExonOverlapBedFile, sep='\t', lineterminator='\n',header=None)
    senseExonOverlapBed.columns = ["queryChr","queryStart","queryEnd","queryGeneId","queryScore","queryStrand","refChr","refStart","refEnd","refGeneId","refScore","refStrand","overlapLength"]
    uniqueGeneIdsOverlappingSenseStrand=set(senseExonOverlapBed.queryGeneId.unique().tolist())
    # write new GTF files 
    sameStrandDeletedGenesGTFFileObj = open(sameStrandDeletedGenesGTFFile, "w")
    cleanedSameStrandExonsGTFFileObj = open(cleanedSameStrandExonsGTFFile, "w")
    # keep counts to cross check the consistancy of lines in the files
    geneCountToBeDelted=0
    geneCountToBeKept=0
    for gendId in LncBookGenes.keys():
        gene=LncBookGenes.get(gendId)
        geneWithTranscriptsAndExonsLines=getGeneWithTranscriptsAndExonsAsLines(gene)
        if gendId in uniqueGeneIdsOverlappingSenseStrand:
            geneCountToBeDelted=geneCountToBeDelted+1;
            sameStrandDeletedGenesGTFFileObj.write(geneWithTranscriptsAndExonsLines)
        else:
            geneCountToBeKept=geneCountToBeKept+1
            cleanedSameStrandExonsGTFFileObj.write(geneWithTranscriptsAndExonsLines)
    print("Number of genes to be deleted, because of exon overlap on the same strand : \t",len(uniqueGeneIdsOverlappingSenseStrand))
    print("Number of genes *actually* deleted, because of exon overlap on the same strand : \t",geneCountToBeDelted)
    if(len(uniqueGeneIdsOverlappingSenseStrand)==geneCountToBeDelted):
        print("Number of genes to be deleted are equal to the deleted ones! Success",len(uniqueGeneIdsOverlappingSenseStrand),"\tand\t",geneCountToBeDelted)
    else:
        sys.exit("-------Error Number of genes to be deleted are equal to the deleted ones--------"+str(len(uniqueGeneIdsOverlappingSenseStrand))+"\tand\t"+str(geneCountToBeDelted))
    return uniqueGeneIdsOverlappingSenseStrand

In [5]:
def getProteinCodingExonsFromIndex(index,df):
    refStart=df.iloc[index]['refStart']
    refEnd=df.iloc[index]['refEnd']
    proteinCodingExons = []
    for i in range(len(refStart)):
        proteinCodingExons.append([refStart[i],refEnd[i]])
    return proteinCodingExons

#getProteinCodingExonsFromIndex(3,df)   

In [6]:
# merge the overlapping proteincoding exons into a long non coding exons
def getUnionOfMultipleranges(proteinCodingExons):
    #proteinCodingExons = [[7, 10], [11, 13], [11, 15], [14, 20], [23, 39]] # should output [[7, 20], [23, 39]]
    b = []
    for begin,end in sorted(proteinCodingExons):
        if b and b[-1][1] >= begin - 1:
            b[-1][1] = max(b[-1][1], end)
        else:
            b.append([begin, end])
    return b

In [7]:
# Python3 implementation to find
# intersecting intervals

# Function to print intersecting
# intervals
def printIntervals(arr1, arr2):
	resultingIntervals = []
    # i and j pointers for arr1
	# and arr2 respectively
	i = j = 0
	
	n = len(arr1)
	m = len(arr2)

	# Loop through all intervals unless one
	# of the interval gets exhausted
	while i < n and j < m:
		
		# Left bound for intersecting segment
		l = max(arr1[i][0], arr2[j][0])
		
		# Right bound for intersecting segment
		r = min(arr1[i][1], arr2[j][1])
		
		# If segment is valid print it
		if l <= r:
			#print('{', l, ',', r, '}')
			resultingIntervals.append([l,r])
            

		# If i-th interval's right bound is
		# smaller increment i else increment j
		if arr1[i][1] < arr2[j][1]:
			i += 1
		else:
			j += 1
	resultingIntervalsDf = pd.DataFrame(resultingIntervals, columns = ['start', 'end'])
	resultingIntervalsDf=resultingIntervalsDf.drop_duplicates() # sometimes same intervals are returned if multiple protein coding exons overlap in the same region
	return resultingIntervalsDf

In [8]:
# +1 is to make sure that equality falls in the range
def updateLncRNAExonSingleCase(resultingIntervalsDf,lncRNAStart,lncRNAEnd):
    updatedLncRNAStart=0
    updatedLncRNAEnd=0
    resultingExons=[]
    row=resultingIntervalsDf.iloc[0]
    eStart=row['start']
    eEnd=row['end']
    if (lncRNAStart in range(eStart-1,eEnd) and lncRNAEnd in range(eStart,eEnd+1)):
        updatedLncRNAStart=0
        updatedLncRNAEnd=0
    elif lncRNAStart < eStart: # Use the orignal lncRNA start
        updatedLncRNAStart=lncRNAStart
        updatedLncRNAEnd=eStart-globalExonBoundaryOffset
    elif lncRNAEnd > eEnd: # Use the orignal lncRNA end
        updatedLncRNAStart=eEnd+globalExonBoundaryOffset
        updatedLncRNAEnd=lncRNAEnd
    resultingExons.append([updatedLncRNAStart,updatedLncRNAEnd])
    return resultingExons

In [9]:
def updateLncRNAExonMultipleCase(resultingIntervalsDf,lncRNAStart,lncRNAEnd):
    updatedLncRNAStart=0
    updatedLncRNAEnd=0
    
    prevStart=0
    prevEnd=0
    resultingMultipleExons = [] 
    ### check if the lncRNA is contained completely inside the overlapping intervals. Meaning we cannot chop it.
    for index, row in resultingIntervalsDf.iterrows():
        eStart=row['start']
        eEnd=row['end']
        if (lncRNAStart in range(eStart-1,eEnd) and lncRNAEnd in range(eStart,eEnd+1)):
            updatedLncRNAStart=0
            updatedLncRNAEnd=0
            resultingMultipleExons.append([updatedLncRNAStart,updatedLncRNAEnd])
            return resultingMultipleExons
    ### If reached here, then chop it off.
    for i in range(len(resultingIntervalsDf)-1):
        currentRow=resultingIntervalsDf.iloc[i]
        nextRow=resultingIntervalsDf.iloc[i+1]
        resultingMultipleExons.append([currentRow['end']+globalExonBoundaryOffset,nextRow['start']-globalExonBoundaryOffset])
    ## Add Start and end if lncRNA is starting and/or ending after the intervals
    minOfStarts=resultingIntervalsDf['start'].min()
    maxOfEnds=resultingIntervalsDf['end'].max()
    if(lncRNAStart<minOfStarts):
         resultingMultipleExons.append([lncRNAStart,minOfStarts-globalExonBoundaryOffset])
    if(lncRNAEnd>maxOfEnds):
         resultingMultipleExons.append([maxOfEnds+globalExonBoundaryOffset,lncRNAEnd])
    
    resultingMultipleExons = sorted(resultingMultipleExons, key=lambda x: x[1])
    return resultingMultipleExons


In [10]:
# Create New Exons. Chop lncRNA exons or assign [0,0] to delete them
def getNewExons(df,minimumExonLength):
    df['proteinCodingExonsMerged'] = np.empty((len(df), 0)).tolist() # For merged protein coding exons
    df['newExonsWithNoLengthLimit'] = np.empty((len(df), 0)).tolist()# add empty column for new exons
    df['newExons'] = np.empty((len(df), 0)).tolist() # add empty column for new exons
    df['numberOfNewExons'] = 0 # add empty column for numberOfNewExons
    for index in range(len(df)):
        proteinCodingExons = getProteinCodingExonsFromIndex(index,df)
        lncRNAStart=df.iloc[index]['queryStart']
        lncRNAEnd=df.iloc[index]['queryEnd']
        lncRNAExon = [ [lncRNAStart , lncRNAEnd ] ]
        # obtain Union of multiple ranges. This will combine all the overlapping protein coding exons 
        # into one long exon (if they have overlapping boundires or just starting 1 nt after each other.)
        proteinCodingExons=getUnionOfMultipleranges(proteinCodingExons)
        df.at[index,'proteinCodingExonsMerged'] = proteinCodingExons
        resultingIntervals=printIntervals(proteinCodingExons, lncRNAExon)
        newExons=[]
        if(len(resultingIntervals)==1):
            #print("-----------------------Single----------------------------")
            newExons=updateLncRNAExonSingleCase(resultingIntervals,lncRNAStart,lncRNAEnd)
        else:
            #print("-------------------------Multiple--------------------------")
            newExons=updateLncRNAExonMultipleCase(resultingIntervals,lncRNAStart,lncRNAEnd)
        df.at[index,'newExonsWithNoLengthLimit'] = newExons
        
        # remove any exons with length less than minimumExonLength
        filteredNewExon=[]
        for exon in newExons:
            start=exon[0]
            end=exon[1]
            if(end-start>=minimumExonLength):
                filteredNewExon.append([start,end])
        
        df.at[index,'newExons'] = filteredNewExon
        df.at[index,'numberOfNewExons'] = len(filteredNewExon)

    df.sort_values(by=['queryGeneId'])
    return df

In [11]:
def testExonChoppingCalculation(proteinCodingExons,lncRNACases):
    for i in range(len(lncRNACases)):
        lncRNAStart=lncRNACases[i][0]
        lncRNAEnd=lncRNACases[i][1]
        lncRNAExon = [ [lncRNAStart , lncRNAEnd ] ]
        resultingIntervals=printIntervals(proteinCodingExons, lncRNAExon)
        if(len(resultingIntervals)==1):
            print("Case ,",i+1,":---------Single--")
            print(updateLncRNAExonSingleCase(resultingIntervals,lncRNAStart,lncRNAEnd))
        else:
            print("Case ,",i+1,":---------Multiple-")
            print(updateLncRNAExonMultipleCase(resultingIntervals,lncRNAStart,lncRNAEnd))
        print("-----------------------------------------------------------------------------")

In [12]:
def updateOrDeleteExonsWithNewExons(gene,exonsToBeUpdatedForGeneDf,minimumExonLength):
    for transcriptID in gene.transcripts:
        transcriptObj=gene.transcripts.get(transcriptID)
        for exonObj in transcriptObj.exons:
            rowFromCsvForExonId=exonsToBeUpdatedForGeneDf.loc[exonsToBeUpdatedForGeneDf['exonId'] == exonObj.identifier]
            if not rowFromCsvForExonId.empty:
                newExons=rowFromCsvForExonId['newExons']
                numberOfNewExons=rowFromCsvForExonId['numberOfNewExons']            
                if numberOfNewExons.all()==0: # is one value in the file but pandas is loading it as series and 0 0
                    # delete the exon from the Transcript. 
                    #print('deleteng in transcriptID\t'+transcriptID+'\texonObj.identifier\t'+exonObj.identifier)
                    transcriptObj.exons = [item for item in transcriptObj.exons if item.identifier != exonObj.identifier]
                    transcriptObj.n_exons=transcriptObj.n_exons-1
                else:
                    firstExon=True;
                    for start,end in itertools.chain.from_iterable(newExons):
                        if firstExon==True:
                            exonObj.start=start
                            exonObj.end=end
                            exonObj.length=abs(exonObj.end - exonObj.start+1)
                            #print('Updating')
                            firstExon=False;
                        else:
                            #print('Adding more exons')
                            # just updated the start,end and id in the copied exon
                            copiedExonObj = copy.deepcopy(exonObj) 
                            copiedExonObj.start=start
                            copiedExonObj.end=end
                            copiedExonObj.length=abs(copiedExonObj.end - copiedExonObj.start+1)
                            copiedExonObj.identifier=copiedExonObj.chromosome +"_" +str(copiedExonObj.start)\
                            +"_" +str(copiedExonObj.end)+"_" +copiedExonObj.strand
                            transcriptObj.add_exon(copiedExonObj) 
                            transcriptObj.n_exons=transcriptObj.n_exons+1

        gene.transcripts[transcriptID]=transcriptObj
    return gene

In [13]:
def updateOrDeleteTranscripts(gene,numberOfTranscriptsDeletedBecauseOfNoExon):
    for transcriptID in list(gene.transcripts):
        transcriptObj=gene.transcripts.get(transcriptID)
        if(transcriptObj.n_exons==0):
            del gene.transcripts[transcriptID] # remove transcript with no exons
            numberOfTranscriptsDeletedBecauseOfNoExon=numberOfTranscriptsDeletedBecauseOfNoExon+1
        else:
            start=sys.maxsize
            end=0
            for exon in transcriptObj.exons:
                if exon.start<start:
                    start=exon.start
                if exon.end>end:
                    end=exon.end
            transcriptObj.start=start
            transcriptObj.end=end
            gene.transcripts[transcriptID]=transcriptObj
    return gene,numberOfTranscriptsDeletedBecauseOfNoExon
def updateOrDeleteGenes(CleanedSameStrandGenes,gene,numberOfGenesDeletedBecauseOfNoTranscripts):
    start=sys.maxsize
    end=0
    if(len(gene.transcripts)==0):
        del CleanedSameStrandGenes[gene.identifier]# remove gene with no transcripts
        numberOfGenesDeletedBecauseOfNoTranscripts=numberOfGenesDeletedBecauseOfNoTranscripts+1
    else:
        for transcriptID in list(gene.transcripts):
            transcriptObj=gene.transcripts.get(transcriptID)
            if transcriptObj.start<start:
                start=transcriptObj.start
            if transcriptObj.end>end:
                end=transcriptObj.end
        gene.start=start
        gene.end=end
        CleanedSameStrandGenes[gene.identifier]=gene  
    return CleanedSameStrandGenes,numberOfGenesDeletedBecauseOfNoTranscripts


        

In [14]:
def getRemainingAntiSenseToBeDeleted(uniqueGeneIdsOverlappingSenseStrand,uniqueGeneIdsOverlappingAntisense,alreadyDeletedInSenseFile):
    print("unique GeneIds Overlapping Sense Strand: ",len(uniqueGeneIdsOverlappingSenseStrand))
    print("unique GeneIds Overlapping Antisense Strand: ",len(uniqueGeneIdsOverlappingAntisense))
    remainingAntiSenseToBeDeleted=set(uniqueGeneIdsOverlappingAntisense) - set(uniqueGeneIdsOverlappingSenseStrand)
    alreadyDeletedInSense = [gene for gene in uniqueGeneIdsOverlappingAntisense if gene in uniqueGeneIdsOverlappingSenseStrand]
    print("Genes that overlap on both sense and anti-sense strand or alreadyDeletedInSense: ",len(alreadyDeletedInSense))
    print("Remaining Anti Sense Genes To Be Deleted: ",len(remainingAntiSenseToBeDeleted))
    if(len(alreadyDeletedInSense)+len(remainingAntiSenseToBeDeleted)!=len(uniqueGeneIdsOverlappingAntisense)):
         sys.exit("Error!!! Already Deleted In Sense and Remaining Anti Sense Genes To Be Deleted are not equal to total \
        number of genes overlapping in antisense",len(alreadyDeletedInSense)+len(remainingAntiSenseToBeDeleted))
    
    print("Success: Already Deleted In Sense and Remaining Anti Sense Genes To Be Deleted are equal to total \
        number of genes overlapping in antisense",len(alreadyDeletedInSense)+len(remainingAntiSenseToBeDeleted))
    ### Write to a text file for a possible future use
    alreadyDeletedInSenseFileObj = open(alreadyDeletedInSenseFile, "w")
    for element in alreadyDeletedInSense:
        alreadyDeletedInSenseFileObj.write(element + "\n")
    alreadyDeletedInSenseFileObj.close()
    return remainingAntiSenseToBeDeleted

In [15]:
def cleaneAntiSenseOverlappingExons(uniqueGeneIdsOverlappingAntisense,CleanedSameStrandGenes):
    nrTotalGenes=len(CleanedSameStrandGenes.keys())
    nrGenesProcessed=0
    nrTranscriptsDelOfNoExon=0
    nrGenesDelOfNoTranscripts=0
    for geneID in uniqueGeneIdsOverlappingAntisense:
        gene=CleanedSameStrandGenes.get(geneID)
        exonsToBeUpdatedForGeneDf=df.loc[df['queryGeneId'] == geneID]
        try:
            nrGenesProcessed=nrGenesProcessed+1
            gene=updateOrDeleteExonsWithNewExons(gene,exonsToBeUpdatedForGeneDf,minimumExonLength)
            gene , nrTranscriptsDelOfNoExon=updateOrDeleteTranscripts(gene,nrTranscriptsDelOfNoExon)
            CleanedSameStrandGenes, nrGenesDelOfNoTranscripts=updateOrDeleteGenes(CleanedSameStrandGenes,gene,nrGenesDelOfNoTranscripts)
        except AttributeError as error:
            sys.exit('Gene causing the erros is\t'+str(geneID))
    print('number Of Total Genes before processing\t'+str(nrTotalGenes))
    print('number Of Genes processed\t'+str(nrGenesProcessed))
    print('nr Transcripts Deleted Because Of No Exon\t'+str(nrTranscriptsDelOfNoExon))  
    print('number Of Genes Deleted Because Of No Transcripts\t'+str(nrGenesDelOfNoTranscripts))
    if(nrGenesProcessed!=len(uniqueGeneIdsOverlappingAntisense)):
        sys.exit("-------Error Number of genes processed and written are not equal to initial-")
    validateAndWriteGTFToFile(CleanedSameStrandGenes,cleanedForSenseAndAntisenseFile)

In [16]:
#### we will take only unique exons within a gene for overlap to reduce the lines and unncessary comparisons. i-e If a gene has multiple transcripts and if the same exon is repeated in these transcripts, we will take that exon only once.
# To insure that we do not mess it with other genes, we will take it inside each gene only. This will take all the exons of the gene and only once even if they are repeating inside that gene in different transcripts (of that gene).
def createExonBedFile(gtfFile,outputDir,filename):
    lines_seen = set() # holds lines already seen
    with open(gtfFile, "r") as a_file:
            for line in a_file:
                if not line.startswith("#"):
                    gtfLine = line.split("\t")
                    if(gtfLine[2]=="exon"):
                        longString=gtfLine[8].split()
                        geneId=longString[1].split(";")[0].replace('"', '')
                        exonLine=gtfLine[0]+"\t"+gtfLine[3]+"\t"+gtfLine[4]+"\t"+ geneId+"\t"+'0'+"\t"+gtfLine[6]+"\n"
                        if exonLine not in lines_seen: # not a duplicate
                            lines_seen.add(exonLine)
    my_file = open(outputDir+filename, "w")
    my_file.writelines(lines_seen)
    my_file.close()
    return(lines_seen)


In [18]:
######## input File Names
# GTFs are downloaded in Download_and_Create_Genome_indices.sh
LncBook_file = "Genomes//reference_sources/LncBook_Version2.0_OnlyLnc_hg38.gtf_corrected.gtf"
genCod_file="Genomes//reference_sources/gencode.v32.primary_assembly.annotation.gtf.protein_coding.gtf"
gtfGeneCodeLncRNA = "Genomes//reference_sources/gencode.v32.primary_assembly.annotation.gtf.gencode_lncRNA.gtf"


######## Output File Names
outputDir="Genomes/OverlapCleansing/"
# LncExpDB overlap with Genecode protein coding
senseExonOverlapBedFile=outputDir+"strand_LnRNA_ExonOverlap_ProteinCodingExons.bed"
antiSenseExonOverlapBedFile=outputDir+"antiStrand_LnRNA_ExonOverlap_ProteinCodingExons.bed"
#  Genecode protein coding overlap with Genecode lncRNA
senseProteinGenecodelncRNABedFile=outputDir+"sense_protein_GenecodelncRNA.bed"
antisenseProteinGenecodelncRNABedFile=outputDir+"antisense_protein_GenecodelncRNA.bed"

#
sameStrandDeletedGenesGTFFile=outputDir+"same_Strand_Deleted_Genes.gtf"
cleanedSameStrandExonsGTFFile=outputDir+"cleaned_Same_Strand_Exons.gtf"
#testCleanedSameStrandExonsGTFFile = outputDir+"testCasesCleanedSameStrandExonsGTF.gtf" # just for testing purpose
alreadyDeletedInSenseFile=outputDir+"alreadyDeletedInSenseFile.txt"

cleanedForSenseAndAntisenseFile=outputDir+"cleanedForSense_and_Antisense.gtf"
antiSenseExonOverlapCompressedBedFile = outputDir+"antiSenseExonOverlapBedFileCompressed.csv"
antiSenseExonsChoppedDfFilePath = outputDir+"antiSenseExonsChoppedDf.csv"

## parameters
globalExonBoundaryOffset=100 # There should be gap of 100 nucleotides from the trimmed boundaries
minimumExonLength=200 # new chopped exons length should be atleast 200nt

In [16]:
print("---lncRNAs----")
LncBookGenes, LncBookTranscripts, LncBookExons=read_gtf_file(LncBook_file)
printNumberOfGeneTranscriptsExons(LncBookGenes, LncBookTranscripts, LncBookExons)
print("---protein coding----")
genCodGenes, genCodTranscripts, genCodExons=read_gtf_file(genCod_file)
printNumberOfGeneTranscriptsExons(genCodGenes, genCodTranscripts, genCodExons)

print("---Gencode lncRNAs----")
gencodeLncGenes, gencodeLncTranscripts, gencodeLncExons=read_gtf_file(gtfGeneCodeLncRNA)
printNumberOfGeneTranscriptsExons(gencodeLncGenes, gencodeLncTranscripts, gencodeLncExons)

---lncRNAs----
geneCount 101285
transcriptCount 331167
exonCount 611102
---protein coding----
geneCount 19384
transcriptCount 151714
exonCount 476299
---Gencode lncRNAs----
geneCount 16849
transcriptCount 47262
exonCount 109075


In [17]:
############### Generate bed files
lines_seen=createExonBedFile(LncBook_file,outputDir,"LncBook_Version2.0_OnlyLnc_hg38_Exons.bed")
print("The length of Uniq exons per gene in LncBook_file is:", len(lines_seen))

lines_seen=createExonBedFile(genCod_file,outputDir,"gencode.v32.primary_assembly.annotation.gtf.filtered_Exons.bed")
print("The length of Uniq exons per gene in genCod_file is:", len(lines_seen))

lines_seen=createExonBedFile(gtfGeneCodeLncRNA,outputDir,"geneCode_lncRNA.bed")
print("The length of Uniq exons per gene in genCodlncRNA_file is:", len(lines_seen))

The length of Uniq exons per gene in LncBook_file is: 611102
The length of Uniq exons per gene in genCod_file is: 476299
The length of Uniq exons per gene in genCodlncRNA_file is: 109075


In [ ]:
##### perform the overlap commands in bash setup.sh 

In [18]:
#### Calculate the numbers here. One lncRNA exon can overlap with multiple protein coding
# we want to count unique lncRNA exons

def printNumberOFUniqueLncRNAsOverlap(bedFilePath):
    exonOverlapBed = pd.read_csv(bedFilePath, sep='\t', lineterminator='\n',header=None)
    exonOverlapBed.columns = ["queryChr","queryStart","queryEnd","queryGeneId","queryScore","queryStrand","refChr","refStart","refEnd","refGeneId","refScore","refStrand","overlapLength"]
    # Query overlap numbers
    uniqueQueryGeneIdsOverlap=str(len(exonOverlapBed.queryGeneId.unique().tolist()))
    queryTotalExonsOverlapDf = exonOverlapBed[["queryChr","queryStart","queryEnd","queryGeneId"]]
    queryUniqueExonsOverlapDf = queryTotalExonsOverlapDf.drop_duplicates()
    
    # Reference overlap numbers
    uniqueRefGeneIdsOverlap=str(len(exonOverlapBed.refGeneId.unique().tolist()))
    refTotalExonsOverlapDf = exonOverlapBed[["refChr","refStart","refEnd","refGeneId"]]
    refUniqueExonsOverlapDf = refTotalExonsOverlapDf.drop_duplicates()
    print("-----------",bedFilePath)
    print("uniqueQueryGeneIdsOverlap: ",uniqueQueryGeneIdsOverlap," uniqueRefGeneIdsOverlap: ",uniqueRefGeneIdsOverlap)
    # Total exon overlap will be the same. as these are just number of lines in the file and not so meaningful
    #print("queryTotalExonsOverlap: ",queryTotalExonsOverlapDf.shape[0]," refTotalExonsOverlap",refTotalExonsOverlapDf.shape[0])
    print("queryUniqueExonsOverlap: ",queryUniqueExonsOverlapDf.shape[0]," refUniqueExonsOverlap",refUniqueExonsOverlapDf.shape[0])
    print("\n")

##################################### Call it for the overlapped bed files 
# Gencode with LncExpDB
print("Query always represent lncRNAs and Reference represents protein coding in the dataframe from the bed file\n\n")
printNumberOFUniqueLncRNAsOverlap(senseExonOverlapBedFile) 
printNumberOFUniqueLncRNAsOverlap(antiSenseExonOverlapBedFile)
# Within Gencode
printNumberOFUniqueLncRNAsOverlap(senseProteinGenecodelncRNABedFile) 
printNumberOFUniqueLncRNAsOverlap(antisenseProteinGenecodelncRNABedFile)


Query always represent lncRNAs and Reference represents protein coding in the dataframe from the bed file


----------- /data/Mullen_1/Raza/scRNA/Genomes/OverlapCleansing/strand_LnRNA_ExonOverlap_ProteinCodingExons.bed
uniqueQueryGeneIdsOverlap:  7531  uniqueRefGeneIdsOverlap:  6309
queryUniqueExonsOverlap:  24357  refUniqueExonsOverlap 42868


----------- /data/Mullen_1/Raza/scRNA/Genomes/OverlapCleansing/antiStrand_LnRNA_ExonOverlap_ProteinCodingExons.bed
uniqueQueryGeneIdsOverlap:  14212  uniqueRefGeneIdsOverlap:  10492
queryUniqueExonsOverlap:  44062  refUniqueExonsOverlap 47057


----------- /data/Mullen_1/Raza/scRNA/Genomes/OverlapCleansing/sense_protein_GenecodelncRNA.bed
uniqueQueryGeneIdsOverlap:  516  uniqueRefGeneIdsOverlap:  619
queryUniqueExonsOverlap:  2106  refUniqueExonsOverlap 3514


----------- /data/Mullen_1/Raza/scRNA/Genomes/OverlapCleansing/antisense_protein_GenecodelncRNA.bed
uniqueQueryGeneIdsOverlap:  3791  uniqueRefGeneIdsOverlap:  3590
queryUniqueExonsOverlap

In [19]:
############################################# Sense overlap Cleaning ###################
# delete LncRNA Genes that Overlap Same Exons in the Same Strand. As they can be un-annotated protein coding genes.
print("-----Cleaning GTF annotation for Sense strand overlap----")
uniqueGeneIdsOverlappingSenseStrand=deleteLncRNAGenesOverlappingSameExonsSameStrand(senseExonOverlapBedFile,sameStrandDeletedGenesGTFFile,cleanedSameStrandExonsGTFFile)


-----Cleaning GTF annotation for Sense strand overlap----
Number of genes to be deleted, because of exon overlap on the same strand : 	 7531
Number of genes *actually* deleted, because of exon overlap on the same strand : 	 7531
Number of genes to be deleted are equal to the deleted ones! Success 7531 	and	 7531


In [20]:
############################################# Anti-Sense overlap Cleaning ###################
#Clean (delete or chop) LncRNA exons that Overlap Exons in the anti-sense Strand. Because cell ranger will assign the reads to + strand (always even if lncRNA is +)
antiSenseExonOverlapBed = pd.read_csv(antiSenseExonOverlapBedFile, sep='\t', lineterminator='\n',header=None)
antiSenseExonOverlapBed.columns = ["queryChr","queryStart","queryEnd","queryGeneId","queryScore","queryStrand","refChr","refStart","refEnd","refGeneId","refScore","refStrand","overlapLength"]
df=antiSenseExonOverlapBed.groupby(['queryChr', 'queryStart','queryEnd','queryGeneId','queryStrand'], as_index = False).agg({'refChr': list,'refStart': list,'refEnd': list,'refGeneId': list,'refStrand': list,'overlapLength': list,})
# create exonId for easy matching in the comparisons
df["exonId"] = df["queryChr"] +"_" + df["queryStart"].astype(str)+"_"+ df["queryEnd"].astype(str)+"_"+ df["queryStrand"]+"_"+ df["queryGeneId"]
antiSenseExonOverlapCompressedBedFileObj = open(antiSenseExonOverlapCompressedBedFile, "w")
#write the compressed exons. Compressed line put multiple overlaps of an exon in one line
df.to_csv(antiSenseExonOverlapCompressedBedFileObj, encoding='utf-8',index=False)
## Get new exon structures for the LncRNAs based on the overlapping information.
# This function internally calls updateLncRNAExonSingleCase and updateLncRNAExonMultipleCase, which in terms also call the interval functions and all.
df=getNewExons(df,minimumExonLength)
# write one line for each exon to chop or delete it from input GTF file (in this case LnCRNA)
antiSenseExonsChoppedDfObj = open(antiSenseExonsChoppedDfFilePath, "w")
df.to_csv(antiSenseExonsChoppedDfObj, encoding='utf-8',index=False)

In [21]:
uniqueGeneIdsOverlappingAntisense=df.queryGeneId.unique().tolist()
print("Number of genes to be deleted or updated, because of exon overlap on the anti-sense strand : \t",len(set(uniqueGeneIdsOverlappingAntisense)))
CleanedSameStrandGenes, CleanedSameStrandTranscripts, CleanedSameStrandExons=read_gtf_file(cleanedSameStrandExonsGTFFile)
printNumberOfGeneTranscriptsExons(CleanedSameStrandGenes, CleanedSameStrandTranscripts, CleanedSameStrandExons)



Number of genes to be deleted or updated, because of exon overlap on the anti-sense strand : 	 14212
geneCount 93754
transcriptCount 311205
exonCount 565717


In [22]:
remainingAntiSenseToBeDeleted=getRemainingAntiSenseToBeDeleted(uniqueGeneIdsOverlappingSenseStrand,uniqueGeneIdsOverlappingAntisense,alreadyDeletedInSenseFile)
print("-----Cleaning GTF annotation for Anti-sense strand overlap----")
cleaneAntiSenseOverlappingExons(remainingAntiSenseToBeDeleted,CleanedSameStrandGenes)



unique GeneIds Overlapping Sense Strand:  7531
unique GeneIds Overlapping Antisense Strand:  14212
Genes that overlap on both sense and anti-sense strand or alreadyDeletedInSense:  849
Remaining Anti Sense Genes To Be Deleted:  13363
Success: Already Deleted In Sense and Remaining Anti Sense Genes To Be Deleted are equal to total         number of genes overlapping in antisense 14212
-----Cleaning GTF annotation for Anti-sense strand overlap----
number Of Total Genes before processing	93754
number Of Genes processed	13363
nr Transcripts Deleted Because Of No Exon	5837
number Of Genes Deleted Because Of No Transcripts	2539


In [7]:
print("..Loading the Cleaned and final GTF file for numbers and validity : \t",cleanedForSenseAndAntisenseFile)
genes, transcripts, exons=read_gtf_file(cleanedForSenseAndAntisenseFile)
printNumberOfGeneTranscriptsExons(genes, transcripts, exons)



..Loading the Cleaned and final GTF file for numbers and validity : 	 /data/Mullen_1/Raza/scRNA/Genomes/OverlapCleansing/cleanedForSense_and_Antisense.gtf
geneCount 91215
transcriptCount 305368
exonCount 537373


In [24]:
# create exons bed file again in a test dir to check the overlap. There should not be any overlap now
testPath= outputDir+"temp/"
if not os.path.exists(testPath):
    os.mkdir(testPath, mode = 0o777, dir_fd = None)



In [25]:
filename = os.path.basename(cleanedForSenseAndAntisenseFile)+str("_OnlyLnc_hg38_Exons.bed")
print(filename)
lines_seen=createExonBedFile(cleanedForSenseAndAntisenseFile,testPath,filename)
print("The length of Uniq exons per gene in LncBook_file is:", len(lines_seen))

cleanedForSense_and_Antisense.gtf_OnlyLnc_hg38_Exons.bed
The length of Uniq exons per gene in LncBook_file is: 537373


In [ ]:
# perform the overlap commands in bash setup.sh for testing overlaps

In [25]:
## How many lncRNA, transcripts and exons we added compared to Gencode lncRNAs
print("---Gencode lncRNAs----")
printNumberOfGeneTranscriptsExons(gencodeLncGenes, gencodeLncTranscripts, gencodeLncExons)
print("---Singletrome lncRNAs----")
printNumberOfGeneTranscriptsExons(genes, transcripts, exons)
print("----- We added the following additional lncRNAs to the GTF compared to the default Gencode lncRNAs----")
print("Genes: ",len(genes)-len(gencodeLncGenes), " Fold:",round(len(genes)/len(gencodeLncGenes),2))
print("Transcripts: ",len(transcripts)-len(gencodeLncTranscripts), " Fold:",round(len(transcripts)/len(gencodeLncTranscripts),2))
print("Exons: ",len(exons)-len(gencodeLncExons), " Fold:",round(len(exons)/len(gencodeLncExons),2))

---Gencode lncRNAs----
geneCount 16849
transcriptCount 47262
exonCount 109075
---Singletrome lncRNAs----
geneCount 91215
transcriptCount 305368
exonCount 537373
----- We added the following additional lncRNAs to the GTF compared to the default Gencode lncRNAs----
Genes:  74366  Fold: 5.41
Transcripts:  258106  Fold: 6.46
Exons:  428298  Fold: 4.93


In [41]:
print("Loading the Cleaned for same strand GTF file with Gencode proteincoding genes: \t","/data/Mullen_1/Raza/scRNA/Genomes//reference_sources/cleanedForSenseStrand_LncBook_Version2.0__gencode.v32.gtf")
g, t, e=read_gtf_file("/data/Mullen_1/Raza/scRNA/Genomes//reference_sources/cleanedForSenseStrand_LncBook_Version2.0__gencode.v32.gtf")
printNumberOfGeneTranscriptsExons(g, t, e)



Loading the Cleaned for same strand GTF file with Gencode proteincoding genes: 	 /data/Mullen_1/Raza/scRNA/Genomes//reference_sources/cleanedForSenseStrand_LncBook_Version2.0__gencode.v32.gtf
geneCount 113138
transcriptCount 462919
exonCount 1042016


In [20]:
####### TEST CASES FOR THE SCRIPT ABOVE. WE HAVE THE TESTS FOR DUMMY AND REAL DATA BELOW.  
# Test dumy data
proteinCodingExons = [ [ 3000, 4000 ], [ 6000, 7000 ],[ 9000, 10000 ]]
lncRNACases=[[3000,4000], # case 1
             [3500,4000],# case 2
             [2000,3500],# case 3
             [9500,11000],# case 4
             [2000,14000],# case 5
             [3500,9500],# case 6
             [2000,9500],# case 7
             [3500,14000]# case 8
]
#testExonChoppingCalculation(proteinCodingExons,lncRNACases) 

# Real test case: chr10 100233231 100233443 HSALNG0080088 0 +
proteinCodingExons = [ [ 100232298,100233371 ], [ 100232305,100233371 ],[ 100233110,100233371 ]]
lncRNACases=[[100233231,100233443]]
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>---------------Real test case----")
#testExonChoppingCalculation(proteinCodingExons,lncRNACases) 
print(">>>>>>>>>>>>>>>>>>>>># case 9 from bug in real data. Produces two exons with same start and same end")
proteinCodingExons = [ [ 1355004,1355345 ], [ 1355244,1355345 ],[ 1355244,1355512 ]]
lncRNACases=[[1355263,1355391]]
#testExonChoppingCalculation(proteinCodingExons,lncRNACases)
print(">>>>>>>>>>>>>>>>>>>>># case 10 from bug in real data. Start> end in the third generated exon")
proteinCodingExons = [ [ 1197093,1197186 ], [ 1197438,1197475 ],[1197438,1197933],[1197438,1197935]]
lncRNACases=[[1196875,1197496]]
testExonChoppingCalculation(proteinCodingExons,lncRNACases)



>>>>>>>>>>>>>>>>>>>>>>>>>>>>---------------Real test case----
>>>>>>>>>>>>>>>>>>>>># case 9 from bug in real data. Produces two exons with same start and same end
>>>>>>>>>>>>>>>>>>>>># case 10 from bug in real data. Start> end in the third generated exon
Case , 1 :---------Multiple-
[[1196875, 1196993], [1197286, 1197338], [1197575, 1197338]]
-----------------------------------------------------------------------------


In [ ]:
######################## Test Run Real Data Cleaning GTF For AntiSense Exons #####################
# Some will be deleted and some will be updated.
# Caution
# 1. Delete transcript if all the exons is/are deleted
# 2. Delete gene if all the transcript is/are deleted
print(testCleanedSameStrandExonsGTFFile)
# This testCasesAntiSenseExonsChoppedDf.csv is manually created to test the script
# Gene 1. No updates, just write
# Gene 2. Found but [0,0], meaning delete this completely
# Gene 3. Update one Exon to one exon
# Gene 4. Update one Exon to multiple exons
# Gene 5. Update multiple exons of a single gene (one Exon to multiple exons)
# Gene 6. Update multiple exons of a single gene (Update some and delete some)
uniqueGeneIdsOverlappingAntisense=df.queryGeneId.unique().tolist()
print("Number of genes to be deleted or updated, because of exon overlap on the anti-sense strand : \t",len(set(uniqueGeneIdsOverlappingAntisense)))

print("---CleanedSameStrandGTF----")
CleanedSameStrandGenes, CleanedSameStrandTranscripts, CleanedSameStrandExons=read_gtf_file(testCleanedSameStrandExonsGTFFile)
printNumberOfGeneTranscriptsExons(CleanedSameStrandGenes, CleanedSameStrandTranscripts, CleanedSameStrandExons)
uniqueGeneIdsOverlappingAntisense=['HSALNG0000227','HSALNG0000002','HSALNG0000055','HSALNG0000060','HSALNG0000070','HSALNG0000137','HSALNG0000141']
cleandAntiSenseOverlappingExons(uniqueGeneIdsOverlappingAntisense,CleanedSameStrandGenes)

